# StormWISE_TMDL JUPITER NOTEBOOK VERSION
## Instructions:
1. Before running StormWISE, you must prepare an input text file in YAML format and you will specify the name of that file in a code block below.

2. We then calculate and display the MAXIMUM POSSIBLE BENEFITS that can be achieved by installing Green Stormwater Infrastructure (GSI)" AT ALL POSSIBLE SITES in the watershed.

3. We then display an estimate of the TOTAL WATERSHED-WIDE INVESTMENT DOLLARS required" to achieve maximum possible benefits"

4. You will set the displayOption variable value to select the option for showing how the maximum investment dollars and maximum possible benefits would be distributed across geographic zones, land uses, and GSI technologies.

5. Then you will be asked to choose numeric values for the RUNOFF LOAD REDUCTIONS (BENEFIT LEVELS) THAT YOU ACTUALLY WANT TO ACHIEVE, using the units specified.

6. StormWISE will then run its OPTIMIZATION MODEL to find the best way to allocate investment dollars among different geographic zones, land uses, and GSI technologies so as to MINIMIZE TOTAL WATERSHED-WIDE INVESTMENT DOLLARS.

7. The StormWISE solution will be displayed to the screen and you will then be given several options for breaking out optimized benefits and investment dollars according to geographic zone, land use, and GSI technologies.

8. You will then be invited to perform Sensitivity Analyses by entering ALTERNATIVE BENEFIT LEVELS to learn how different runoff load reductions change the total investments required and the distribution of benefits by zones, land uses and GSI technologies"

### Set the inYamlFile variable which contains the parameters for the watershed you are modeling:

In [3]:
import os
import sys
import yaml
from copy import deepcopy
from IPython.display import display, Markdown
import pandas as pd
from StormWISE_TMDL_Engine.stormwise_tmdl import stormwise
from StormWISE_TMDL_Engine.stormwise_tmdl import evaluate_solution
from StormWISE_TMDL_Engine.stormwise_tmdl_benefits_and_bounds import benefit_slopes
from StormWISE_TMDL_Engine.stormwise_tmdl_benefits_and_bounds import upper_bounds
from StormWISE_TMDL_Engine.stormwise_tmdl_benefits_and_bounds import convert_benefit_units
from StormWISE_TMDL_Engine.stormwise_tmdl_benefits_and_bounds import format_and_convert_benefit_dict
from Arts_Python_Tools.tools import multiply_dict_by_constant
from Arts_Python_Tools.tools import format_dict_as_strings

# Set amplPath when switching between Linux and Windows:
#amplPath = "ampl.mswin64/ampl"  # note: you must also set the solver path in stormwise_tmdl.run
amplPath = "ampl.linux64/ampl"   # note: you must also set the solver path in stormwise_tmdl.run

with open('convert_benefits.yaml', 'r') as fin:
    convertBenefits = yaml.load(fin)
benefitUnits = convertBenefits['benefitUnits']
#print(convertBenefits)
benefitConvertUnits = convertBenefits['benefitConvertUnits']
########
inYamlFile = "wingohocking.yaml"
########
with open(inYamlFile, 'r') as fin:
    inYamlDict = yaml.load(fin)
#print(inYamlDict)

### Code for print_benefits_investments()

In [4]:
def print_benefits_investments(solutionDict,benefitUnits,benefitConvertUnits,displayOption):
    benTotsByBenefit = solutionDict['benTotsByBenefit']
    displayDict = format_and_convert_benefit_dict(benTotsByBenefit,"%0.2f",benefitConvertUnits,benefitUnits)
    display(Markdown("**Totals for Each Benefit:**"))
    df = pd.DataFrame.from_dict(displayDict,'index')
    display(df)
    investmentTotal = solutionDict['investmentTotal']
    investmentTotalMillions = investmentTotal/1e6
    display(Markdown("Total GSI Investment Required to Obtain These Benefits:   **%0.2f Million**\n" % investmentTotalMillions))
    if displayOption == 1:
        showDict = deepcopy(solutionDict['benTotsByBenefitByZone'])
        displayDict = format_and_convert_benefit_dict(showDict,"%0.2f",benefitConvertUnits,benefitUnits)
        display(Markdown("**Benefits** By Zone:"))
        df = pd.DataFrame.from_dict(displayDict,'index')
        display(df)
        display(Markdown('<br>'))
        showDict = deepcopy(solutionDict['invTotsByZone'])
        multiply_dict_by_constant(showDict,1e-6)  # convert to $Million
        format_dict_as_strings(showDict,"$ %0.2f Million")
        display(Markdown("**Investment Dollars** By Zone:"))
        df = pd.DataFrame.from_dict(showDict,'index')
        display(df)
    elif displayOption == 2:
        showDict = deepcopy(solutionDict['benTotsByBenefitByLanduse'])
        displayDict = format_and_convert_benefit_dict(showDict,"%0.2f",benefitConvertUnits,benefitUnits)
        display(Markdown("**Benefits** By Land Use:"))
        df = pd.DataFrame.from_dict(displayDict,'index')
        display(df)
        display(Markdown('<br>'))
        showDict = deepcopy(solutionDict['invTotsByLanduse'])
        multiply_dict_by_constant(showDict,1e-6)  # convert to $Million
        format_dict_as_strings(showDict,"$ %0.2f Million")
        display(Markdown("**Investment Dollars** By Land Use:"))
        df = pd.DataFrame.from_dict(showDict,'index')
        display(df)
    elif displayOption == 3:    
        showDict = deepcopy(solutionDict['benTotsByBenefitByGi'])
        displayDict = format_and_convert_benefit_dict(showDict,"%0.2f",benefitConvertUnits,benefitUnits)
        display(Markdown("**Benefits** By GSI Technology:"))
        df = pd.DataFrame.from_dict(displayDict,'index')
        display(df)
        display(Markdown('<br>'))
        showDict = deepcopy(solutionDict['invTotsByGi'])
        multiply_dict_by_constant(showDict,1e-6)  # convert to $Million
        format_dict_as_strings(showDict,"$ %0.2f Million")
        display(Markdown("**Investment Dollars** By GSI Technology:"))
        df = pd.DataFrame.from_dict(showDict,'index')
        display(df)
    elif displayOption == 4:
        showDict = deepcopy(solutionDict['benTotsByBenefitByZoneByLanduse'])
        displayDict = format_and_convert_benefit_dict(showDict,"%0.2f",benefitConvertUnits,benefitUnits)
        display(Markdown("**Benefits** By Zone AND By Landuse::"))
        for t in displayDict:            
            display(Markdown("<br>For Benefit: **%s**:" % t))
            df = pd.DataFrame.from_dict(displayDict[t],'index')
            display(df)
        display(Markdown('<br>'))
        showDict = deepcopy(solutionDict['invTotsByZoneByLanduse'])
        multiply_dict_by_constant(showDict,1e-6)  # convert to $Million
        format_dict_as_strings(showDict,"$%0.2f Million")
        display(Markdown("**Investment Dollars** By Zone AND By Landuse:"))
        df = pd.DataFrame.from_dict(showDict,'index')
        display(df)
    elif displayOption == 5:
        showDict = deepcopy(solutionDict['benTotsByBenefitByZoneByGi'])
        displayDict = format_and_convert_benefit_dict(showDict,"%0.2f",benefitConvertUnits,benefitUnits)
        display(Markdown("**Benefits** By Zone AND By GSI Technology:"))
        for t in displayDict:            
            display(Markdown("<br>For Benefit: **%s**:" % t))
            df = pd.DataFrame.from_dict(displayDict[t],'index')
            display(df)
        display(Markdown('<br>'))
        showDict = deepcopy(solutionDict['invTotsByZoneByGi'])
        multiply_dict_by_constant(showDict,1e-6)  # convert to $Million
        format_dict_as_strings(showDict,"$%0.2f Million")
        display(Markdown("**Investment Dollars** By Zone AND By GSI Technology:"))
        df = pd.DataFrame.from_dict(showDict,'index')
        display(df)
    elif displayOption == 6:
        showDict = deepcopy(solutionDict['benTotsByBenefitByLanduseByGi'])
        displayDict = format_and_convert_benefit_dict(showDict,"%0.2f",benefitConvertUnits,benefitUnits)
        display(Markdown("**Benefits** By Land Use AND By GSI Technology:"))
        for t in displayDict:            
            display(Markdown("<br>For Benefit: **%s**:" % t))
            df = pd.DataFrame.from_dict(displayDict[t],'index')
            display(df)
        display(Markdown('<br>'))
        showDict = deepcopy(solutionDict['invTotsByLanduseByGi'])
        multiply_dict_by_constant(showDict,1e-6)  # convert to $Million
        format_dict_as_strings(showDict,"$%0.2f Million")
        display(Markdown("**Investment Dollars** By Land Use AND By GSI Technology:"))
        df = pd.DataFrame.from_dict(showDict,'index')
        display(df)
    elif displayOption == 7:
        showDict = deepcopy(solutionDict['benefitsByZoneByLanduseByGi'])
        displayDict = format_and_convert_benefit_dict(showDict,"%0.4f",benefitConvertUnits,benefitUnits)
        display(Markdown("Benefits By Zone AND By Land Use AND By GSI Technology:"))
        for t in displayDict:            
            display(Markdown("<br>For Benefit: **%s**:" % t))
            oneBenefitDisplayDict = displayDict[t]
            for i in oneBenefitDisplayDict:
                display(Markdown("<br>For Land Use: **%s**:" % i))
                df = pd.DataFrame.from_dict(oneBenefitDisplayDict[i],'index')
                display(df)
        display(Markdown('<br>'))
        showDict = deepcopy(solutionDict['decisions'])
        multiply_dict_by_constant(showDict,1e-6)  # convert to $Million
        format_dict_as_strings(showDict,"$%0.4f Million")
        display(Markdown("**Investment Dollars** By Zone AND By Land Use AND By GSI Technology:"))
        for i in showDict:            
            display(Markdown("\nFor Land Use: **%s**:" % i))
            df = pd.DataFrame.from_dict(showDict[i],'index')
            display(df)
            display(Markdown('<br>'))
    else:
        print("SORRY: your value for the variable displayOption is not valid - TRY AGAIN")

### Values of displayOption variable for INVESTMENT and BENEFIT details:

|displayOption|How to Display|
|---|---|
|1|By geographic zone|
|2|By land use|
|3|By green infrastructure technology|
|4|By zone and by land use"|
|5|By zone and by green infrastructure technology|
|6|By land use and by green infrastructure technology|
|7|By zone and by land use and by green infrastructure technology|

### Set the Display Option and Generate Upper Limits on Benefits & Investments:

In [5]:
displayOption = 3
s = benefit_slopes(inYamlDict)
T = inYamlDict['T']
upperBounds = upper_bounds(inYamlDict)
upperBoundSolutionDict = evaluate_solution(upperBounds,s,inYamlDict)
display(Markdown("\n\n**UPPER LIMITS ON BENEFITS & INVESTMENTS:**\n"))
print_benefits_investments(upperBoundSolutionDict,benefitUnits,benefitConvertUnits,displayOption)  



**UPPER LIMITS ON BENEFITS & INVESTMENTS:**


**Totals for Each Benefit:**

,0
1_volume (Million Gallons),693.36
2_sediment (Tons),844.35
3_nitrogen (Pounds),9207.16
4_phosphorous (Pounds),1167.61


Total GSI Investment Required to Obtain These Benefits:   **257.46 Million**


**Benefits** By GSI Technology:

,1_bioswale,2_rain_garden,3_tree_trench,4_rain_barrel,5_impervious_removal,6_permeable_pavement,7_green_roof
1_volume (Million Gallons),34.24,194.20,139.12,134.78,36.26,63.22,91.53
2_sediment (Tons),154.79,204.23,179.75,194.41,47.96,56.89,6.33
3_nitrogen (Pounds),438.31,1851.55,1480.69,2139.33,638.79,1086.19,1572.31
4_phosphorous (Pounds),108.54,258.98,206.69,255.32,78.35,105.93,153.80


<br>

**Investment Dollars** By GSI Technology:

,0
1_bioswale,$ 6.77 Million
2_rain_garden,$ 8.24 Million
3_tree_trench,$ 13.66 Million
4_rain_barrel,$ 17.50 Million
5_impervious_removal,$ 10.72 Million
6_permeable_pavement,$ 65.62 Million
7_green_roof,$ 134.95 Million


### Set minimum benefit levels and run the StormWISE model in a loop:

In [6]:
while True:
    benefitDict = {}
    display(Markdown("### Enter Your Required Minimum Runoff Load Reductions in Specified Units:"))
    tDict = {}
    tDictStrings = {}
    upperBounds = upper_bounds(inYamlDict)
    upperBoundSolutionDict = evaluate_solution(upperBounds,s,inYamlDict)
    maxBenefitDict = upperBoundSolutionDict['benTotsByBenefit']
    #print(maxBenefitDict)
    for t in T:
        maxBenefit = (maxBenefitDict[t]*benefitConvertUnits[t])
        maxValStr = "{0:.2f}".format(maxBenefit)
        prompt = "%s %s (max value = %s):  " % (t,benefitUnits[t],maxValStr)
        inString = input(prompt)
        tDictStrings[t] = inString
        tDict[t] = float(inString)/benefitConvertUnits[t]  # convert to fundamental units

    benefitDict['benefitLowerBounds'] = tDict
    display(Markdown("## Generating Results for Benefit Levels:"))
    df = pd.DataFrame.from_dict(tDictStrings,'index')
    display(df)
    display(Markdown("### Running StormWISE using AMPL with MINOS solver"))
    # running the AMPL executable from the stormwise() function
    decisions = stormwise(amplPath,inYamlDict,benefitDict)
    solutionDict = evaluate_solution(decisions,s,inYamlDict)
    print_benefits_investments(solutionDict,benefitUnits,benefitConvertUnits,displayOption)
    
    inString = input("Do you want to run StormWISE again with different benefit levels? Y or N: ")

    if inString == 'N' or inString == 'n':
        print("StormWISE Run Completed")
        break

### Enter Your Required Minimum Runoff Load Reductions in Specified Units:

1_volume Million Gallons (max value = 693.36):  600
2_sediment Tons (max value = 844.35):  800
3_nitrogen Pounds (max value = 9207.16):  9200
4_phosphorous Pounds (max value = 1167.61):  1100


## Generating Results for Benefit Levels:

,0
1_volume,600
2_sediment,800
3_nitrogen,9200
4_phosphorous,1100


### Running StormWISE using AMPL with MINOS solver

...

,1_bioswale,2_rain_garden,3_tree_trench,4_rain_barrel,5_impervious_removal,6_permeable_pavement,7_green_roof
1_volume (Million Gallons),0.00,0.00,0.00,0.00,0.00,0.00,0.00
2_sediment (Tons),0.00,0.00,0.00,0.00,0.00,0.00,0.00
3_nitrogen (Pounds),0.00,0.00,0.00,0.00,0.00,0.00,0.00
4_phosphorous (Pounds),0.00,0.00,0.00,0.00,0.00,0.00,0.00


<br>

**Investment Dollars** By GSI Technology:

,0
1_bioswale,$ 0.00 Million
2_rain_garden,$ 0.00 Million
3_tree_trench,$ 0.00 Million
4_rain_barrel,$ 0.00 Million
5_impervious_removal,$ 0.00 Million
6_permeable_pavement,$ 0.00 Million
7_green_roof,$ 0.00 Million


Do you want to run StormWISE again with different benefit levels? Y or N: n
StormWISE Run Completed
